In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')
import pickle

In [2]:
matches = pd.read_csv('matches.csv')
matches

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2
0,335982,2007/08,Bangalore,18-04-2008,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,2007/08,Chandigarh,19-04-2008,League,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,241.0,20.0,N,NaN,MR Benson,SL Shastri
2,335984,2007/08,Delhi,19-04-2008,League,MF Maharoof,Feroz Shah Kotla,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,130.0,20.0,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,2007/08,Mumbai,20-04-2008,League,MV Boucher,Wankhede Stadium,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,166.0,20.0,N,NaN,SJ Davis,DJ Harper
4,335986,2007/08,Kolkata,20-04-2008,League,DJ Hussey,Eden Gardens,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5.0,111.0,20.0,N,NaN,BF Bowden,K Hariharan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,1359544,2023,Bengaluru,21-05-2023,League,Shubman Gill,"M Chinnaswamy Stadium, Bengaluru",Royal Challengers Bangalore,Gujarat Titans,Gujarat Titans,field,Gujarat Titans,wickets,6.0,198.0,20.0,N,NaN,Nitin Menon,VK Sharma
1020,1370350,2023,Chennai,23-05-2023,Qualifier 1,RD Gaikwad,"MA Chidambaram Stadium, Chepauk, Chennai",Chennai Super Kings,Gujarat Titans,Gujarat Titans,field,Chennai Super Kings,runs,15.0,173.0,20.0,N,NaN,AK Chaudhary,CB Gaffaney
1021,1370351,2023,Chennai,24-05-2023,Eliminator,Akash Madhwal,"MA Chidambaram Stadium, Chepauk, Chennai",Mumbai Indians,Lucknow Super Giants,Mumbai Indians,bat,Mumbai Indians,runs,81.0,183.0,20.0,N,NaN,BNJ Oxenford,VK Sharma
1022,1370352,2023,Ahmedabad,26-05-2023,Qualifier 2,Shubman Gill,"Narendra Modi Stadium, Ahmedabad",Gujarat Titans,Mumbai Indians,Mumbai Indians,field,Gujarat Titans,runs,62.0,234.0,20.0,N,NaN,Nitin Menon,RJ Tucker


In [3]:
deliveries = pd.read_csv('deliveries.csv')
deliveries

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,extra_runs,total_runs,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,SC Ganguly,P Kumar,BB McCullum,0,1,1,legbyes,0,NaN,NaN,NaN
1,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
2,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,BB McCullum,P Kumar,SC Ganguly,0,1,1,wides,0,NaN,NaN,NaN
3,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,4,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
4,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,5,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243812,1370353,2,Chennai Super Kings,Gujarat Titans,14,2,S Dube,MM Sharma,RA Jadeja,1,0,1,NaN,0,NaN,NaN,NaN
243813,1370353,2,Chennai Super Kings,Gujarat Titans,14,3,RA Jadeja,MM Sharma,S Dube,1,0,1,NaN,0,NaN,NaN,NaN
243814,1370353,2,Chennai Super Kings,Gujarat Titans,14,4,S Dube,MM Sharma,RA Jadeja,1,0,1,NaN,0,NaN,NaN,NaN
243815,1370353,2,Chennai Super Kings,Gujarat Titans,14,5,RA Jadeja,MM Sharma,S Dube,6,0,6,NaN,0,NaN,NaN,NaN


In [4]:
matches['team1'].unique()

array(['Royal Challengers Bangalore', 'Kings XI Punjab',
       'Delhi Daredevils', 'Mumbai Indians', 'Kolkata Knight Riders',
       'Rajasthan Royals', 'Deccan Chargers', 'Chennai Super Kings',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Sunrisers Hyderabad',
       'Gujarat Lions', 'Rising Pune Supergiants',
       'Rising Pune Supergiant', 'Delhi Capitals', 'Punjab Kings',
       'Lucknow Super Giants', 'Gujarat Titans'], dtype=object)

In [5]:
teams = [
    'Royal Challengers Bangalore',
    'Punjab Kings',
    'Delhi Capitals',
    'Mumbai Indians',
    'Kolkata Knight Riders',
    'Rajasthan Royals',
    'Chennai Super Kings',
    'Sunrisers Hyderabad',
    'Lucknow Super Giants',
    'Gujarat Titans'
]

In [6]:
matches['team1'] = matches['team1'].str.replace('Kings XI Punjab', 'Punjab Kings')
matches['team2'] = matches['team1'].str.replace('Kings XI Punjab', 'Punjab Kings')

matches['team1'] = matches['team1'].str.replace('Delhi Daredevils', 'Delhi Capitals')
matches['team2'] = matches['team1'].str.replace('Delhi Daredevils', 'Delhi Capitals')

matches['team1'] = matches['team1'].str.replace('Deccan Chargers', 'Sunrisers Hyderabad')
matches['team2'] = matches['team1'].str.replace('Deccan Chargers', 'Sunrisers Hyderabad')

In [7]:
matches = matches[matches['team1'].isin(teams)]
matches = matches[matches['team2'].isin(teams)]

In [8]:
matches

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2
0,335982,2007/08,Bangalore,18-04-2008,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Royal Challengers Bangalore,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,2007/08,Chandigarh,19-04-2008,League,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Punjab Kings,Punjab Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,241.0,20.0,N,NaN,MR Benson,SL Shastri
2,335984,2007/08,Delhi,19-04-2008,League,MF Maharoof,Feroz Shah Kotla,Delhi Capitals,Delhi Capitals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,130.0,20.0,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,2007/08,Mumbai,20-04-2008,League,MV Boucher,Wankhede Stadium,Mumbai Indians,Mumbai Indians,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,166.0,20.0,N,NaN,SJ Davis,DJ Harper
4,335986,2007/08,Kolkata,20-04-2008,League,DJ Hussey,Eden Gardens,Kolkata Knight Riders,Kolkata Knight Riders,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5.0,111.0,20.0,N,NaN,BF Bowden,K Hariharan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,1359544,2023,Bengaluru,21-05-2023,League,Shubman Gill,"M Chinnaswamy Stadium, Bengaluru",Royal Challengers Bangalore,Royal Challengers Bangalore,Gujarat Titans,field,Gujarat Titans,wickets,6.0,198.0,20.0,N,NaN,Nitin Menon,VK Sharma
1020,1370350,2023,Chennai,23-05-2023,Qualifier 1,RD Gaikwad,"MA Chidambaram Stadium, Chepauk, Chennai",Chennai Super Kings,Chennai Super Kings,Gujarat Titans,field,Chennai Super Kings,runs,15.0,173.0,20.0,N,NaN,AK Chaudhary,CB Gaffaney
1021,1370351,2023,Chennai,24-05-2023,Eliminator,Akash Madhwal,"MA Chidambaram Stadium, Chepauk, Chennai",Mumbai Indians,Mumbai Indians,Mumbai Indians,bat,Mumbai Indians,runs,81.0,183.0,20.0,N,NaN,BNJ Oxenford,VK Sharma
1022,1370352,2023,Ahmedabad,26-05-2023,Qualifier 2,Shubman Gill,"Narendra Modi Stadium, Ahmedabad",Gujarat Titans,Gujarat Titans,Mumbai Indians,field,Gujarat Titans,runs,62.0,234.0,20.0,N,NaN,Nitin Menon,RJ Tucker


In [9]:
# required columns
# battting team, bowling team, city, venue, runs left, balls left, wickets left, total runs (1st innings), current run rate, required run rate, winner

In [10]:
first_inning = deliveries[deliveries['inning'] == 1]

In [11]:
total_score = first_inning.groupby(['match_id', 'inning']).sum()['total_runs'].reset_index()
total_score['total_runs_first_inning'] = total_score['total_runs']
total_score.drop(['total_runs'], inplace=True, axis=1)
total_score

,match_id,inning,total_runs_first_inning
0,335982,1,222
1,335983,1,240
2,335984,1,129
3,335985,1,165
4,335986,1,110
...,...,...,...
1019,1359544,1,197
1020,1370350,1,172
1021,1370351,1,182
1022,1370352,1,233


In [12]:
# merge_df = runs.merge(total_score[['match_id', 'total_runs_first_inning']], on='match_id')
# merge_df

In [13]:
match_df = matches.merge(total_score[['match_id', 'total_runs_first_inning']], left_on='id', right_on='match_id')
match_df

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,...,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2,match_id,total_runs_first_inning
0,335982,2007/08,Bangalore,18-04-2008,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Royal Challengers Bangalore,Royal Challengers Bangalore,...,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen,335982,222
1,335983,2007/08,Chandigarh,19-04-2008,League,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Punjab Kings,Punjab Kings,Chennai Super Kings,...,runs,33.0,241.0,20.0,N,NaN,MR Benson,SL Shastri,335983,240
2,335984,2007/08,Delhi,19-04-2008,League,MF Maharoof,Feroz Shah Kotla,Delhi Capitals,Delhi Capitals,Rajasthan Royals,...,wickets,9.0,130.0,20.0,N,NaN,Aleem Dar,GA Pratapkumar,335984,129
3,335985,2007/08,Mumbai,20-04-2008,League,MV Boucher,Wankhede Stadium,Mumbai Indians,Mumbai Indians,Mumbai Indians,...,wickets,5.0,166.0,20.0,N,NaN,SJ Davis,DJ Harper,335985,165
4,335986,2007/08,Kolkata,20-04-2008,League,DJ Hussey,Eden Gardens,Kolkata Knight Riders,Kolkata Knight Riders,Deccan Chargers,...,wickets,5.0,111.0,20.0,N,NaN,BF Bowden,K Hariharan,335986,110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959,1359544,2023,Bengaluru,21-05-2023,League,Shubman Gill,"M Chinnaswamy Stadium, Bengaluru",Royal Challengers Bangalore,Royal Challengers Bangalore,Gujarat Titans,...,wickets,6.0,198.0,20.0,N,NaN,Nitin Menon,VK Sharma,1359544,197
960,1370350,2023,Chennai,23-05-2023,Qualifier 1,RD Gaikwad,"MA Chidambaram Stadium, Chepauk, Chennai",Chennai Super Kings,Chennai Super Kings,Gujarat Titans,...,runs,15.0,173.0,20.0,N,NaN,AK Chaudhary,CB Gaffaney,1370350,172
961,1370351,2023,Chennai,24-05-2023,Eliminator,Akash Madhwal,"MA Chidambaram Stadium, Chepauk, Chennai",Mumbai Indians,Mumbai Indians,Mumbai Indians,...,runs,81.0,183.0,20.0,N,NaN,BNJ Oxenford,VK Sharma,1370351,182
962,1370352,2023,Ahmedabad,26-05-2023,Qualifier 2,Shubman Gill,"Narendra Modi Stadium, Ahmedabad",Gujarat Titans,Gujarat Titans,Mumbai Indians,...,runs,62.0,234.0,20.0,N,NaN,Nitin Menon,RJ Tucker,1370352,233


In [14]:
match_df = match_df[['match_id', 'venue', 'winner', 'total_runs_first_inning']]
match_df

,match_id,venue,winner,total_runs_first_inning
0,335982,M Chinnaswamy Stadium,Kolkata Knight Riders,222
1,335983,"Punjab Cricket Association Stadium, Mohali",Chennai Super Kings,240
2,335984,Feroz Shah Kotla,Delhi Daredevils,129
3,335985,Wankhede Stadium,Royal Challengers Bangalore,165
4,335986,Eden Gardens,Kolkata Knight Riders,110
...,...,...,...,...
959,1359544,"M Chinnaswamy Stadium, Bengaluru",Gujarat Titans,197
960,1370350,"MA Chidambaram Stadium, Chepauk, Chennai",Chennai Super Kings,172
961,1370351,"MA Chidambaram Stadium, Chepauk, Chennai",Mumbai Indians,182
962,1370352,"Narendra Modi Stadium, Ahmedabad",Gujarat Titans,233


In [15]:
match_df = match_df.merge(deliveries, on='match_id')
match_df

,match_id,venue,winner,total_runs_first_inning,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,extra_runs,total_runs,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder
0,335982,M Chinnaswamy Stadium,Kolkata Knight Riders,222,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,SC Ganguly,P Kumar,BB McCullum,0,1,1,legbyes,0,NaN,NaN,NaN
1,335982,M Chinnaswamy Stadium,Kolkata Knight Riders,222,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
2,335982,M Chinnaswamy Stadium,Kolkata Knight Riders,222,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,BB McCullum,P Kumar,SC Ganguly,0,1,1,wides,0,NaN,NaN,NaN
3,335982,M Chinnaswamy Stadium,Kolkata Knight Riders,222,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,4,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
4,335982,M Chinnaswamy Stadium,Kolkata Knight Riders,222,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,5,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229688,1370353,"Narendra Modi Stadium, Ahmedabad",Chennai Super Kings,214,2,Chennai Super Kings,Gujarat Titans,14,2,S Dube,MM Sharma,RA Jadeja,1,0,1,NaN,0,NaN,NaN,NaN
229689,1370353,"Narendra Modi Stadium, Ahmedabad",Chennai Super Kings,214,2,Chennai Super Kings,Gujarat Titans,14,3,RA Jadeja,MM Sharma,S Dube,1,0,1,NaN,0,NaN,NaN,NaN
229690,1370353,"Narendra Modi Stadium, Ahmedabad",Chennai Super Kings,214,2,Chennai Super Kings,Gujarat Titans,14,4,S Dube,MM Sharma,RA Jadeja,1,0,1,NaN,0,NaN,NaN,NaN
229691,1370353,"Narendra Modi Stadium, Ahmedabad",Chennai Super Kings,214,2,Chennai Super Kings,Gujarat Titans,14,5,RA Jadeja,MM Sharma,S Dube,6,0,6,NaN,0,NaN,NaN,NaN


In [16]:
match_df = match_df[match_df['inning'] == 2].reset_index()

In [17]:
match_df

,index,match_id,venue,winner,total_runs_first_inning,inning,batting_team,bowling_team,over,ball,...,bowler,non_striker,batsman_runs,extra_runs,total_runs,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder
0,124,335982,M Chinnaswamy Stadium,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,1,...,AB Dinda,W Jaffer,1,0,1,NaN,0,NaN,NaN,NaN
1,125,335982,M Chinnaswamy Stadium,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,2,...,AB Dinda,R Dravid,0,1,1,wides,0,NaN,NaN,NaN
2,126,335982,M Chinnaswamy Stadium,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,3,...,AB Dinda,R Dravid,0,0,0,NaN,0,NaN,NaN,NaN
3,127,335982,M Chinnaswamy Stadium,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,4,...,AB Dinda,R Dravid,1,0,1,NaN,0,NaN,NaN,NaN
4,128,335982,M Chinnaswamy Stadium,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,5,...,AB Dinda,W Jaffer,1,0,1,NaN,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110772,229688,1370353,"Narendra Modi Stadium, Ahmedabad",Chennai Super Kings,214,2,Chennai Super Kings,Gujarat Titans,14,2,...,MM Sharma,RA Jadeja,1,0,1,NaN,0,NaN,NaN,NaN
110773,229689,1370353,"Narendra Modi Stadium, Ahmedabad",Chennai Super Kings,214,2,Chennai Super Kings,Gujarat Titans,14,3,...,MM Sharma,S Dube,1,0,1,NaN,0,NaN,NaN,NaN
110774,229690,1370353,"Narendra Modi Stadium, Ahmedabad",Chennai Super Kings,214,2,Chennai Super Kings,Gujarat Titans,14,4,...,MM Sharma,RA Jadeja,1,0,1,NaN,0,NaN,NaN,NaN
110775,229691,1370353,"Narendra Modi Stadium, Ahmedabad",Chennai Super Kings,214,2,Chennai Super Kings,Gujarat Titans,14,5,...,MM Sharma,S Dube,6,0,6,NaN,0,NaN,NaN,NaN


In [18]:
runs = match_df.groupby('match_id')['total_runs'].apply(np.cumsum).reset_index()
match_df['current_score'] = runs['total_runs']
runs

,match_id,level_1,total_runs
0,335982,0,1
1,335982,1,2
2,335982,2,2
3,335982,3,3
4,335982,4,4
...,...,...,...
110772,1370353,110772,159
110773,1370353,110773,160
110774,1370353,110774,161
110775,1370353,110775,167


In [19]:
match_df['runs_left'] = match_df['total_runs_first_inning'] - match_df['current_score']

In [20]:
match_df.sample()

,index,match_id,venue,winner,total_runs_first_inning,inning,batting_team,bowling_team,over,ball,...,batsman_runs,extra_runs,total_runs,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder,current_score,runs_left
86266,178988,1237178,Sheikh Zayed Stadium,Sunrisers Hyderabad,131,2,Sunrisers Hyderabad,Royal Challengers Bangalore,16,3,...,0,0,0,NaN,0,NaN,NaN,NaN,99,32


In [21]:
match_df['balls_left'] = 120 - np.where(match_df['over'] != 0, match_df['over'] * match_df['ball'], match_df['ball'])

In [22]:
match_df['crr'] = (match_df['current_score'] * 6)/(120 - match_df['balls_left'])

In [23]:
match_df['rrr'] = (match_df['runs_left'] *6)/ match_df['balls_left']

In [24]:
wickets = match_df.groupby('match_id')['is_wicket'].apply(np.cumsum).values
match_df['wickets_left'] = 10 - wickets
match_df

,index,match_id,venue,winner,total_runs_first_inning,inning,batting_team,bowling_team,over,ball,...,is_wicket,player_dismissed,dismissal_kind,fielder,current_score,runs_left,balls_left,crr,rrr,wickets_left
0,124,335982,M Chinnaswamy Stadium,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,1,...,0,NaN,NaN,NaN,1,221,119,6.000000,11.142857,10
1,125,335982,M Chinnaswamy Stadium,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,2,...,0,NaN,NaN,NaN,2,220,118,6.000000,11.186441,10
2,126,335982,M Chinnaswamy Stadium,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,3,...,0,NaN,NaN,NaN,2,220,117,4.000000,11.282051,10
3,127,335982,M Chinnaswamy Stadium,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,4,...,0,NaN,NaN,NaN,3,219,116,4.500000,11.327586,10
4,128,335982,M Chinnaswamy Stadium,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,5,...,0,NaN,NaN,NaN,4,218,115,4.800000,11.373913,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110772,229688,1370353,"Narendra Modi Stadium, Ahmedabad",Chennai Super Kings,214,2,Chennai Super Kings,Gujarat Titans,14,2,...,0,NaN,NaN,NaN,159,55,92,34.071429,3.586957,5
110773,229689,1370353,"Narendra Modi Stadium, Ahmedabad",Chennai Super Kings,214,2,Chennai Super Kings,Gujarat Titans,14,3,...,0,NaN,NaN,NaN,160,54,78,22.857143,4.153846,5
110774,229690,1370353,"Narendra Modi Stadium, Ahmedabad",Chennai Super Kings,214,2,Chennai Super Kings,Gujarat Titans,14,4,...,0,NaN,NaN,NaN,161,53,64,17.250000,4.968750,5
110775,229691,1370353,"Narendra Modi Stadium, Ahmedabad",Chennai Super Kings,214,2,Chennai Super Kings,Gujarat Titans,14,5,...,0,NaN,NaN,NaN,167,47,50,14.314286,5.640000,5


In [25]:
def win(x):
    return 1 if x['batting_team'] == x['winner'] else 0

In [26]:
match_df['result'] = match_df.apply(win, axis=1)

In [27]:
match_df = match_df[['venue', 'batting_team', 'bowling_team', 'total_runs_first_inning', 'runs_left', 'balls_left', 'wickets_left', 'crr', 'rrr', 'result']]

In [28]:
final_df = match_df[match_df['balls_left'] != 0]
final_df

,venue,batting_team,bowling_team,total_runs_first_inning,runs_left,balls_left,wickets_left,crr,rrr,result
0,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,222,221,119,10,6.000000,11.142857,0
1,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,222,220,118,10,6.000000,11.186441,0
2,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,222,220,117,10,4.000000,11.282051,0
3,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,222,219,116,10,4.500000,11.327586,0
4,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,222,218,115,10,4.800000,11.373913,0
...,...,...,...,...,...,...,...,...,...,...
110772,"Narendra Modi Stadium, Ahmedabad",Chennai Super Kings,Gujarat Titans,214,55,92,5,34.071429,3.586957,1
110773,"Narendra Modi Stadium, Ahmedabad",Chennai Super Kings,Gujarat Titans,214,54,78,5,22.857143,4.153846,1
110774,"Narendra Modi Stadium, Ahmedabad",Chennai Super Kings,Gujarat Titans,214,53,64,5,17.250000,4.968750,1
110775,"Narendra Modi Stadium, Ahmedabad",Chennai Super Kings,Gujarat Titans,214,47,50,5,14.314286,5.640000,1


In [29]:
x = final_df.iloc[:, :-1]
y = final_df.iloc[:, -1]

In [30]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [31]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((88596, 9), (22149, 9), (88596,), (22149,))

In [32]:
ctf = ColumnTransformer([
    ('ctf', OneHotEncoder(sparse_output=False, drop='first'),['venue', 'batting_team', 'bowling_team'])
],remainder='passthrough')

In [33]:
ctf_data_train = ctf.fit_transform(x_train)
ctf_data_test = ctf.fit_transform(x_test)

In [34]:
param_grid = [    
    {'penalty' : ['l1', 'l2'],
    'solver' : ['lbfgs','newton-cg','liblinear'],
    'max_iter' : [5000]
    }
]

In [35]:
clf = GridSearchCV(LogisticRegression(), param_grid = param_grid, cv = 3, verbose=True, n_jobs=-1)

In [36]:
best_clf = clf.fit(ctf_data_test, y_test)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


In [37]:
best_clf.best_estimator_

LogisticRegression(max_iter=5000)

In [38]:
lr = LogisticRegression(penalty='l1', solver='liblinear', class_weight = 'balanced')

In [39]:
pipe = Pipeline(steps = [('ColumnTransform', ctf),('StandardScaler', StandardScaler()), ('LogisticRegression', lr)])

In [40]:
pipe.fit(x_train, y_train)

Pipeline(steps=[('ColumnTransform',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ctf',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['venue', 'batting_team',
                                                   'bowling_team'])])),
                ('StandardScaler', StandardScaler()),
                ('LogisticRegression',
                 LogisticRegression(class_weight='balanced', penalty='l1',
                                    solver='liblinear'))])

In [41]:
y_pred = pipe.predict(x_test)

In [42]:
pipe.predict_proba(x_test)[12]

array([0.35738658, 0.64261342])

In [43]:
score = accuracy_score(y_test, y_pred)
score

0.7777326290125965

In [44]:
pickle.dump(pipe, open('pipe.pkl', 'wb'))

In [45]:
teams

['Royal Challengers Bangalore',
 'Punjab Kings',
 'Delhi Capitals',
 'Mumbai Indians',
 'Kolkata Knight Riders',
 'Rajasthan Royals',
 'Chennai Super Kings',
 'Sunrisers Hyderabad',
 'Lucknow Super Giants',
 'Gujarat Titans']

In [46]:
final_df['venue'].unique()

array(['M Chinnaswamy Stadium',
       'Punjab Cricket Association Stadium, Mohali', 'Feroz Shah Kotla',
       'Wankhede Stadium', 'Eden Gardens', 'Sawai Mansingh Stadium',
       'Rajiv Gandhi International Stadium, Uppal',
       'MA Chidambaram Stadium, Chepauk', 'Dr DY Patil Sports Academy',
       'Newlands', "St George's Park", 'Kingsmead', 'SuperSport Park',
       'Buffalo Park', 'New Wanderers Stadium', 'De Beers Diamond Oval',
       'OUTsurance Oval', 'Brabourne Stadium',
       'Sardar Patel Stadium, Motera', 'Barabati Stadium',
       'Brabourne Stadium, Mumbai',
       'Vidarbha Cricket Association Stadium, Jamtha',
       'Himachal Pradesh Cricket Association Stadium',
       'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
       'Subrata Roy Sahara Stadium',
       'Shaheed Veer Narayan Singh International Stadium',
       'JSCA International Stadium Complex', 'Sheikh Zayed Stadium',
       'Sharjah Cricket Stadium', 'Dubai International Cricket Stadium',
      